# NOAA challenge for benchmarking RTS

First you need to create to dataset using the original NOAA dataset with the utiliy function: 

```
python3 noaa/noaa_dot_to_coord.py -img path/to/the/images -dot /path/to/the/dotted/images -out /path/to/the/output/folder
```
The function above, create a .txt file for each image file containing the bounding boxes and class for each sea lion.


```
python3 noaa/noaa_create_model_ds.py -i /path/to/the/images -l /path/to/the/bboxes -o /path/to/the/output/folder -n 10
```

The function above create a dataset of sub images and labels for each images. The labels are the total count of sea lion for each class (adult male, sub adult male, female, ...).



# Model Training

Using the helper function load a model and train it on the newly create dataset.

In [ ]:
import train_model
import matplotlib.pyplot as plt
TRAIN_PATH = "/PATH/TO/THE/DATASET"
TRAIN_PATH = "/media/remy/PHILIPS UFD/Model/Train"
EPOCHS = 50

DEVICE = 'cpu'

model = train_model.get_model(DEVICE)

history, model = train_model.train(TRAIN_PATH, model, EPOCHS, DEVICE)

plt.plot(history["train_loss"])
plt.plot(history["test_loss"])
plt.show()

/home/remy/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] initializing the ResNet50 model...
haha


0epochs [00:00, ?epochs/s]

# Model Evaluation

Now we can do the evaluation of the model on the validation set.

In [ ]:
TRAIN_PATH = "/PATH/TO/THE/DATASET"
TRAIN_PATH = "/media/remy/PHILIPS UFD/Model/Test"

print(train_model.eval(TRAIN_PATH, model))